In [2]:
import numpy as np
import networkx as nx

def cal_all_pairs_minimax_path_matrix_by_algo_4(distance_matrix):
    N = len(distance_matrix)
    all_pairs_minimax_matrix = np.zeros((N, N))

    # Construct Minimum Spanning Tree (MST) from the graph
    MST = construct_MST_from_graph(distance_matrix)
    MST_edge_list = list(MST.edges(data='weight'))

    edge_node_list = [(edge[0], edge[1]) for edge in MST_edge_list]
    edge_weight_list = [edge[2] for edge in MST_edge_list]

    # Sort edges in descending order of weight
    edge_large_to_small_arg = np.argsort(edge_weight_list)[::-1]
    edge_weight_large_to_small = np.array(edge_weight_list)[edge_large_to_small_arg]
    edge_nodes_large_to_small = [edge_node_list[i] for i in edge_large_to_small_arg]

    # Iteratively remove edges and calculate tree nodes
    for i, edge_nodes in enumerate(edge_nodes_large_to_small):
        edge_weight = edge_weight_large_to_small[i]
        MST.remove_edge(*edge_nodes)

        tree1_nodes = list(nx.dfs_preorder_nodes(MST, source=edge_nodes[0]))
        tree2_nodes = list(nx.dfs_preorder_nodes(MST, source=edge_nodes[1]))

        for p1 in tree1_nodes:
            for p2 in tree2_nodes:
                all_pairs_minimax_matrix[p1, p2] = edge_weight
                all_pairs_minimax_matrix[p2, p1] = edge_weight

    return all_pairs_minimax_matrix

def construct_MST_from_graph(distance_matrix):
    G = nx.Graph()
    N = len(distance_matrix)
    for i in range(N):
        for j in range(i + 1, N):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    MST = nx.minimum_spanning_tree(G)
    return MST


In [3]:
def variant_of_Floyd_Warshall(adj_matrix):
    p = adj_matrix.copy()
    N = len(adj_matrix)

    for i in range(N):
        for j in range(N):
            if i != j:
                for k in range(N):
                    if i != k and j != k:
                        p[j, k] = min(p[j, k], max(p[j, i], p[i, k]))
    return p


In [7]:
import numpy as np
import time

# Utility function to create random adjacency matrices for undirected weighted graphs
def generate_random_graph(num_nodes, max_weight=100):
    adj_matrix = np.random.randint(1, max_weight + 1, size=(num_nodes, num_nodes))
    adj_matrix = np.triu(adj_matrix, 1)  # Keep only upper triangle
    adj_matrix += adj_matrix.T  # Make it symmetric for undirected graphs
    return adj_matrix

# Check correctness by comparing the two implementations
def check_correctness(adj_matrix):
    algo4_result = cal_all_pairs_minimax_path_matrix_by_algo_4(adj_matrix)
    fw_result = variant_of_Floyd_Warshall(adj_matrix)
    return np.allclose(algo4_result, fw_result)

# Measure execution time for a single test
def measure_execution_time(adj_matrix):
    start_algo4 = time.time()
    cal_all_pairs_minimax_path_matrix_by_algo_4(adj_matrix)
    end_algo4 = time.time()

    start_fw = time.time()
    variant_of_Floyd_Warshall(adj_matrix)
    end_fw = time.time()

    return end_algo4 - start_algo4, end_fw - start_fw

# Run tests with varying graph sizes
def run_tests():
    test_sizes = [5, 10, 20, 50, 100]  # Number of nodes in the graph
    results = []

    for size in test_sizes:
        print(f"Testing with graph size: {size}")
        adj_matrix = generate_random_graph(size)
        
        # Correctness check
        correct = check_correctness(adj_matrix)
        print(f"  Correctness: {'Pass' if correct else 'Fail'}")

        # Execution time measurement
        algo4_times = []
        fw_times = []
        t = 5  # Number of times to measure execution time

        for _ in range(t):
            algo4_time, fw_time = measure_execution_time(adj_matrix)
            algo4_times.append(algo4_time)
            fw_times.append(fw_time)

        # Calculate geometric mean of execution times
        algo4_geo_mean = np.exp(np.mean(np.log(algo4_times)))
        fw_geo_mean = np.exp(np.mean(np.log(fw_times)))

        print(f"  Execution Time (seconds) - Algorithm 4: {algo4_geo_mean:.4f}, Floyd-Warshall: {fw_geo_mean:.4f}")

        # Store results
        results.append({
            "size": size,
            "correct": correct,
            "algo4_time": algo4_geo_mean,
            "fw_time": fw_geo_mean
        })

        # Store results
        results.append({
            "size": size,
            "correct": correct,
            "algo4_time": algo4_time,
            "fw_time": fw_time
        })

    return results

# Run and display test results
test_results = run_tests()
print("\nSummary:")
for result in test_results:
    print(f"Graph size: {result['size']}, Correct: {result['correct']}, "
          f"Algo4 Time: {result['algo4_time']:.4f}s, FW Time: {result['fw_time']:.4f}s")


Testing with graph size: 5
  Correctness: Pass
  Execution Time (seconds) - Algorithm 4: 0.0001, Floyd-Warshall: 0.0000
Testing with graph size: 10
  Correctness: Pass
  Execution Time (seconds) - Algorithm 4: 0.0004, Floyd-Warshall: 0.0006
Testing with graph size: 20
  Correctness: Pass
  Execution Time (seconds) - Algorithm 4: 0.0006, Floyd-Warshall: 0.0035
Testing with graph size: 50
  Correctness: Pass
  Execution Time (seconds) - Algorithm 4: 0.0034, Floyd-Warshall: 0.0592
Testing with graph size: 100
  Correctness: Pass
  Execution Time (seconds) - Algorithm 4: 0.0113, Floyd-Warshall: 0.4747

Summary:
Graph size: 5, Correct: True, Algo4 Time: 0.0001s, FW Time: 0.0000s
Graph size: 5, Correct: True, Algo4 Time: 0.0001s, FW Time: 0.0001s
Graph size: 10, Correct: True, Algo4 Time: 0.0004s, FW Time: 0.0006s
Graph size: 10, Correct: True, Algo4 Time: 0.0002s, FW Time: 0.0004s
Graph size: 20, Correct: True, Algo4 Time: 0.0006s, FW Time: 0.0035s
Graph size: 20, Correct: True, Algo4 Time: